In [14]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import to_categorical


In [15]:
df=pd.read_csv('fer2013.csv')
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [16]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 35
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y= to_categorical(train_y, num_classes=num_labels)
test_y= to_categorical(test_y, num_classes=num_labels)

In [17]:

X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [18]:
##designing the cnn

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))


model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 46, 46, 64)        640       
                                                                 
 conv2d_13 (Conv2D)          (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 22, 22, 64)        0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 22, 22, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 20, 20, 64)        36928     
                                                                 
 conv2d_15 (Conv2D)          (None, 18, 18, 64)        36928     
                                                      

In [21]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [22]:
#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/35


449/449 [==============================] - 82s 179ms/step - loss: 1.7354 - accuracy: 0.2907 - val_loss: 1.5751 - val_accuracy: 0.3795
Epoch 2/35
449/449 [==============================] - 94s 209ms/step - loss: 1.5167 - accuracy: 0.4049 - val_loss: 1.3893 - val_accuracy: 0.4622
Epoch 3/35
449/449 [==============================] - 113s 252ms/step - loss: 1.4016 - accuracy: 0.4597 - val_loss: 1.3822 - val_accuracy: 0.4714
Epoch 4/35
449/449 [==============================] - 94s 210ms/step - loss: 1.3415 - accuracy: 0.4810 - val_loss: 1.3181 - val_accuracy: 0.4896
Epoch 5/35
449/449 [==============================] - 93s 207ms/step - loss: 1.2925 - accuracy: 0.5027 - val_loss: 1.2296 - val_accuracy: 0.5252
Epoch 6/35
449/449 [==============================] - 99s 220ms/step - loss: 1.2559 - accuracy: 0.5200 - val_loss: 1.2124 - val_accuracy: 0.5322
Epoch 7/35
449/449 [==============================] - 104s 233ms/step - loss: 1.2211 - accuracy: 0.5332 - val_loss: 1.2135 - va

In [23]:
#Saving the  model to
fer_json = model.to_json()
with open("livemodel.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("livemodel.h5")
